# Minimizing Structural Vibrations via Guided Diffusion Design Optimization 

This notebook enables the quick and easy generation of novel plate designs with minimized structural vibrations. You can specify for what frequency range you want to generate plate designs with low vibration energy.

IMPORTANT: This notebook does not implement the numerical simulation of the actual vibrations given the plate design. Thus, results displayed here are only based on the deep learning regression model. These results are expected to be biased towards lower values than numerically simulated vibrations.

## Set up environment

In [ ]:
import torch, os
import matplotlib.pyplot as plt
import glob
import numpy as np
import seaborn as sns

plt.rcParams.update({'font.size': 10})
figsize = (6, 4.5)

%reload_ext autoreload
%autoreload 2


In [ ]:
import os, argparse, time, torch
import numpy as np
from acousticnn.plate.configs.main_dir import main_dir
from torchinfo import summary

from diffusers import DDPMScheduler
from diffusion_plate_optim.utils.guidance_utils import diffusion_guidance, load_regression_model, load_diffusion_model, \
    get_moments_from_npz, init_train_logger, get_velocity_sum, print_log

## Generate novel plate designs

In the following cell, we specify the pretrained diffusion and regression models and the number of denoising steps as well as the loss function. 

You can change the loss function to generate plate designs optimized for different excitation frequencies. 

You could for example change the values for min_freq and max freq. Or arbitrarily define another loss function. 

Keep in mind that only integer values between 0 and 300 are valid.

In [ ]:
diffusion_path = "../data/checkpoint_diffusion_model.pt"
regression_path = "../data/checkpoint_full_data.pt"
n_steps = 500
batch_size = 4

min_freq, max_freq = 100, 200

def loss_function_wrapper(min_freq, max_freq):
    def loss_function(predictions):
        subset_predictions = predictions[:, min_freq:max_freq]
        return subset_predictions.sum() * np.sqrt(100/len(subset_predictions.flatten())), len(subset_predictions.flatten())
    return loss_function

loss_function = loss_function_wrapper(min_freq=min_freq, max_freq=max_freq)

In [ ]:
out_mean, out_std, field_mean, field_std = get_moments_from_npz("../data/moments.npz")
diffusion_model = load_diffusion_model(diffusion_path)
summary(diffusion_model)
noise_scheduler = DDPMScheduler(num_train_timesteps=n_steps)
regression_model = load_regression_model(regression_path)
image_snapshots = diffusion_guidance(diffusion_model, regression_model, noise_scheduler, field_mean, field_std, loss_fn=loss_function, n_steps=n_steps, do_diffusion=True, batch_size=batch_size)

# sorting and saving
images, predictions = image_snapshots[-1][0], image_snapshots[-1][1]
sort_idx = np.argsort([get_velocity_sum(pred) for pred in predictions])
images, predictions = images[sort_idx], predictions[sort_idx]

## Visualize Results

In [ ]:
distribution = predictions[:, min_freq:max_freq].mean(1)
distribution_sorted = distribution[np.argsort(distribution)]
images_sorted = images[np.argsort(distribution)]
predictions_sorted = predictions[np.argsort(distribution)]

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
for r in predictions_sorted[:16]:
    plt.plot(r, lw=1)
sns.despine(offset=5)
ax.set_ylim(-20, 80)
ax.grid(which="major", lw=0.2)
ax.set_xlabel('Frequency')
ax.set_ylabel('Amplitude')
plt.show()
plt.close()

In [ ]:
all_responses = np.load("../data/comparison_responses.npz")["arr_0"]
fig, ax = plt.subplots(figsize=figsize)
velocity_distribution = (all_responses[:, min_freq:max_freq]).mean(1)
ax.hist(velocity_distribution, bins='auto',  density=True, edgecolor="black", lw=0.5, label="Training data")
ax.scatter(distribution, y=np.zeros(len(distribution))+0.005, color="red", label="Design results", s=1)
ax.set_ylabel('Density')
plt.legend(frameon=False)
sns.despine(offset=5)


In [ ]:
idx = np.argmin(velocity_distribution)
print(velocity_distribution[idx])

fig, ax = plt.subplots(figsize=figsize)
plt.plot(predictions_sorted[0], lw=0.5, label="Best generated", color="#55a78c")
plt.plot(all_responses[idx], lw=0.5, label=f"best in training data", color="black", linestyle='dashed',)
sns.despine(offset=5)
ax.set_ylim(-20, 80)
ax.grid(which="major", lw=0.2)
ax.set_xlabel('Frequency')
ax.set_ylabel('Amplitude')
plt.legend()
plt.show()
plt.close()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(5.5, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images_sorted[i][0], cmap='gray',vmin=0, vmax=1)
    ax.axis('off')
    ax.set_title(f"{distribution_sorted[i]:4.2f}")
plt.tight_layout()
plt.show()
fig, ax = plt.subplots(figsize=figsize)
plt.imshow(images_sorted[0][0], cmap='gray',vmin=0, vmax=1)
plt.axis('off')
plt.tight_layout()